In [1]:
from platform import python_version
# !pip install shap==0.39
# !pip install transformers==4.5
python_version()

'3.7.13'

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline
from Huggingface.utils import HuggingfaceDatasetManager, DatasetTypeEnum, explain_text, TransformersModelTypeEnum
import torch

In [4]:
torch.cuda.is_available()

True

In [5]:
model_type = TransformersModelTypeEnum.CH_FAKE_TWEET_DETECT
model_name = model_type.value['NAME']
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name).cuda()
pipeline = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True, binary_output=True, device=0)
embedding_dim = model.distilbert.embeddings.word_embeddings.embedding_dim

## Load the respective dataset

In [6]:
dataset_type = DatasetTypeEnum.CHINHON_FAKE_TWEET_DETECT
dataset_manager = HuggingfaceDatasetManager(dataset_type=dataset_type, embedding_dim=embedding_dim)

Loading instances from dir: /home/sersery/Desktop/TUM Informatik/SS22/Thesis/Code/Explainability_of_FND_Models/Huggingface/data/Chinhon_FakeTweetDetect/real_50k.csv
Loaded 50000 instances from /home/sersery/Desktop/TUM Informatik/SS22/Thesis/Code/Explainability_of_FND_Models/Huggingface/data/Chinhon_FakeTweetDetect/real_50k.csv
Loading instances from dir: /home/sersery/Desktop/TUM Informatik/SS22/Thesis/Code/Explainability_of_FND_Models/Huggingface/data/Chinhon_FakeTweetDetect/troll_50k.csv
Loaded 50000 instances from /home/sersery/Desktop/TUM Informatik/SS22/Thesis/Code/Explainability_of_FND_Models/Huggingface/data/Chinhon_FakeTweetDetect/troll_50k.csv


In [7]:
true_samples = dataset_manager.fetch_true_samples(sample_count=10)
true_sample_shap_values = explain_text(model_type=model_type, pipeline=pipeline, text=true_samples[0])

Getting the following indexes: [ 6032 42722 11538 20880 18770 43840 23312  8652 32347 47353]
Explaining the following text: 
<--------------------------------------------------------------------------------------------->
Abandoning our allies and enabling brutality in cities like Aleppo will forever stand as Barack Obamas legacy
<--------------------------------------------------------------------------------------------->
Predicted True with score: 0.7160639762878418
Predicted Fake with score: 0.2839360237121582


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
